Настоящее исследование проводится по заказу крупной телекоммуникационной компании, предоставляющей услуги на территории всего СНГ. Основная цель исследования — определить текущий уровень потребительской лояльности (NPS) среди клиентов из России.

**Цель Исследования**
Определить уровень NPS (Net Promoter Score) среди российских клиентов компании, что позволит оценить их лояльность и выявить ключевые группы пользователей, влияющие на общие показатели.

**Задачи Исследования**
- Подключение к базе данных. Подключиться к базе данных SQLite, содержащей результаты опроса.
- Выгрузка данных. Сформировать витрину данных для дашборда с использованием SQL-запросов.
- Создание дашборда. Построить интерактивный дашборд в Tableau, отображающий уровень NPS и его зависимость от различных пользовательских признаков.
- Анализ данных. Ответить на ключевые вопросы по распределению участников опроса и выявить группы пользователей с различными уровнями лояльности.

**Выводы и Наблюдения**
- Возраст и Пол: Наибольшее количество участников опроса приходится на возрастную группу 30-35 лет, с преобладанием женщин. Активность участников уменьшается с возрастом.
- Тип Пользователей: Среди участников опроса 82.87% составляют старые пользователи, и только 17.13% — новые.
- Городская Активность: Наиболее активные пользователи проживают в Москве, Санкт-Петербурге и Новосибирске.
- Лояльность Пользователей: 52.55% пользователей относятся к группе сторонников, 30.66% — к критикам, и 16.79% — к нейтралам.
- Общий NPS: Общий показатель NPS равен 7.5 из 10, что свидетельствует о высоком уровне лояльности.
- Портрет Сторонников: Большинство сторонников сосредоточено в возрастных группах 25-54 лет, среди которых преобладают женщины, проживающие в крупных городах.
- Города с Высокой Лояльностью: Наивысший уровень лояльности пользователей наблюдается в городах Иваново, Череповец и Саранск.
- Возрастные Группы: Наивысший уровень лояльности пользователей наблюдается в возрастных группах 55-64 и 66+, в то время как наименьший уровень лояльности у пользователей в возрасте 16-24 лет.
- Пол и Новый Пользователь: Уровень лояльности выше среди женщин по сравнению с мужчинами. Лояльность также выше среди новых пользователей по сравнению со старыми.
- Продолжительность Использования: Наивысший уровень лояльности наблюдается у пользователей, которые пользуются сервисом менее одного месяца. Средний уровень лояльности среди всех опрошенных пользователей составляет 21,90.
- Потребление Трафика: Большинство пользователей потребляют трафик в диапазоне от 1 до 15 ГБ, с пиком в сегменте 1-5 ГБ.
- Операционные Системы: Большинство пользователей используют операционную систему Android, с значительно меньшим количеством пользователей на iOS и других системах.
- Устройства: Подавляющее большинство пользователей используют смартфоны, с значительно меньшим количеством пользователей на планшетах и других устройствах.

# Загрузка данных

In [ ]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [ ]:
path_to_db_local = '/content/drive/MyDrive/Colab Notebooks/Обучение/Яндекс Практикум/Аналитик данных (расширенный)/Dataset/telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [ ]:
query = """
SELECT
    u.user_id,
    u.lt_day,
    CASE
        WHEN u.lt_day <= 365 THEN 1
        ELSE 0
    END AS is_new,
    u.age,
    CASE
        WHEN u.gender_segment = 1 THEN 'Женщина'
        WHEN u.gender_segment = 0 THEN 'Мужчина'
        WHEN u.gender_segment IS NULL THEN 'Неизвестно'
        ELSE 'Неизвестно'
    END AS gender_segment,
    u.os_name,
    u.cpe_type_name,
    l.country,
    l.city,
    a.title AS age_segment,
    t.title AS traffic_segment,
    lt.title AS lifetime_segment,
    u.nps_score,
    CASE
        WHEN u.nps_score >= 9 THEN 'promoters'
        WHEN u.nps_score >= 7 THEN 'passives'
        ELSE 'detractors'
    END AS nps_group
FROM
    user u
JOIN
    location l ON u.location_id = l.location_id
JOIN
    age_segment a ON u.age_gr_id = a.age_gr_id
JOIN
    traffic_segment t ON u.tr_gr_id = t.tr_gr_id
JOIN
    lifetime_segment lt ON u.lt_gr_id = lt.lt_gr_id;
"""

In [ ]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,0,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,0,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,0,44.0,Женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,0,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoters
5,A005O0,5501,0,42.0,Женщина,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,detractors
6,A0061R,1236,0,45.0,Мужчина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,promoters
7,A009KS,313,1,35.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,promoters
8,A00AES,3238,0,36.0,Женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,promoters
9,A00F70,4479,0,54.0,Женщина,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,promoters


In [ ]:
df.describe()

,lt_day,is_new,age,nps_score
count,502493.000000,502493.000000,501939.000000,502493.000000
mean,1868.841439,0.171346,39.621946,7.508562
std,1683.701762,0.376811,11.188249,3.020378
min,-21.000000,0.000000,10.000000,1.000000
25%,533.000000,0.000000,31.000000,5.000000
50%,1239.000000,0.000000,38.000000,9.000000
75%,3064.000000,0.000000,47.000000,10.000000
max,9162.000000,1.000000,89.000000,10.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  int64  
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(3), object(10)
memory usage: 53.7+ MB


In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Презентация доступна по ссылке: https://public.tableau.com/views/2_17226365750670/sheet0?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link